Pipeline을 모르는 dummers' code

In [13]:
# !pip install catboost

In [14]:
import pandas as pd
import numpy as np

# 1. Dataset

In [15]:
from catboost.datasets import titanic

train, test = titanic()
print(train.shape, test.shape)
train.head()

(891, 12) (418, 11)


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [16]:
### 결측값이 있는 컬럼명 출력 ###
[col for col in train.columns if train[col].isna().sum() > 0]

['Age', 'Cabin', 'Embarked']

1. 다양한 형태의 컬럼 (숫자, 문자)
2. 다양한 정보 (ID, cardinality, parsible, ... )
3. 결측 (숫자결측, 문자결측)

# 2. 1st draft 모델링

In [17]:
from sklearn.ensemble import RandomForestClassifier

In [18]:
ycol = ["Survived"]
xcol = [col for col in train.columns if col not in ycol]

In [19]:
X = train[xcol]
y = train[ycol]

In [20]:
model = RandomForestClassifier()
model.fit(X, y)

ValueError: could not convert string to float: 'Braund, Mr. Owen Harris'

# 3. One-Hot encoding

In [26]:
from sklearn.preprocessing import OneHotEncoder

In [27]:
num_cols = X.select_dtypes(include=np.number).columns.tolist()
cat_cols = [col for col in X.columns if col not in num_cols]

In [28]:
enc = OneHotEncoder(sparse=False)
enc.fit(X[cat_cols])

/opt/homebrew/Caskroom/miniforge/base/envs/MLOps/lib/python3.8/site-packages/sklearn/preprocessing/_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


OneHotEncoder(sparse=False, sparse_output=False)

In [29]:
new_cat_X = enc.transform(X[cat_cols])
new_X = np.concatenate([new_cat_X, X[num_cols].to_numpy()], axis=1)
pd.DataFrame(new_X)

,0,1,2,3,4,5,6,7,8,9,...,1722,1723,1724,1725,1726,1727,1728,1729,1730,1731
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,3.0,22.0,1.0,0.0,7.2500
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,2.0,1.0,38.0,1.0,0.0,71.2833
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,3.0,3.0,26.0,0.0,0.0,7.9250
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,4.0,1.0,35.0,1.0,0.0,53.1000
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,5.0,3.0,35.0,0.0,0.0,8.0500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,887.0,2.0,27.0,0.0,0.0,13.0000
887,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,888.0,1.0,19.0,0.0,0.0,30.0000
888,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,889.0,3.0,NaN,1.0,2.0,23.4500
889,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,890.0,1.0,26.0,0.0,0.0,30.0000


In [30]:
model = RandomForestClassifier()
model.fit(new_X, y)

ValueError: Input X contains NaN.
RandomForestClassifier does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values

# 4. 결측값 처리

In [31]:
from sklearn.impute import SimpleImputer

In [32]:
imp = SimpleImputer()
imp.fit(new_X)

SimpleImputer()

In [33]:
new_X2 = imp.transform(new_X)

In [34]:
model = RandomForestClassifier()
model.fit(new_X2, y)

/opt/homebrew/Caskroom/miniforge/base/envs/MLOps/lib/python3.8/site-packages/sklearn/base.py:1152: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


RandomForestClassifier()

In [35]:
fitted_value = model.predict(new_X2)

In [36]:
from sklearn.metrics import accuracy_score

In [37]:
accuracy_score(fitted_value, y)

1.0

In [38]:
test_pred = model.predict(test[xcol])
accuracy_score(test_pred, y)

/opt/homebrew/Caskroom/miniforge/base/envs/MLOps/lib/python3.8/site-packages/sklearn/base.py:458: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(


ValueError: could not convert string to float: 'Kelly, Mr. James'

In [39]:
test_cat = enc.transform(test[cat_cols])

ValueError: Found unknown categories ['Compton, Mrs. Alexander Taylor (Mary Eliza Ingersoll)', 'Samaan, Mr. Elias', 'Rosenbaum, Miss. Edith Louise', 'Jefferys, Mr. Ernest Wilfred', 'Duquemin, Mr. Joseph', 'Kimball, Mrs. Edwin Nelson Jr (Gertrude Parsons)', 'Lamb, Mr. John Joseph', 'Julian, Mr. Henry Forbes', 'Assaf, Mr. Gerios', 'Dodge, Dr. Washington', 'Ostby, Miss. Helene Ragnhild', 'Wenzel, Mr. Linhart', 'Sage, Mrs. John (Annie Bullen)', 'Holthen, Mr. Johan Martin', 'Rheims, Mr. George Alexander Lucien', 'Lingane, Mr. John', 'Klasen, Miss. Gertrud Emilia', 'Brown, Miss. Edith Eileen', 'Nilsson, Miss. Berta Olivia', 'Wilkes, Mrs. James (Ellen Needs)', 'Dean, Mrs. Bertram (Eva Georgetta Light)', 'Lane, Mr. Patrick', 'Cotterill, Mr. Henry Harry""', 'Peter, Master. Michael J', 'Spinner, Mr. Henry John', 'Gibson, Mrs. Leonard (Pauline C Boeson)', 'Parker, Mr. Clifford Richard', 'Wirz, Mr. Albert', 'Moore, Mr. Clarence Bloomfield', 'Katavelas, Mr. Vassilios (Catavelas Vassilios")"', 'Sten

# 6. Name, ticket 컬럼 삭제

In [40]:
cat_cols = [col for col in X.columns if col not in num_cols + ['Name','Ticket','Cabin']]

# 7. 전처리 다시 #

In [41]:
enc = OneHotEncoder(sparse=False)
enc.fit(X[cat_cols])

/opt/homebrew/Caskroom/miniforge/base/envs/MLOps/lib/python3.8/site-packages/sklearn/preprocessing/_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


OneHotEncoder(sparse=False, sparse_output=False)

In [42]:
new_new_X = np.concatenate([enc.transform(X[cat_cols]), X[num_cols].to_numpy()], axis=1)

In [43]:
imp = SimpleImputer()
imp.fit(new_new_X)

SimpleImputer()

In [44]:
new_new_new_X = imp.transform(new_new_X)

# 8. 2nd draft 모델링

In [45]:
model = RandomForestClassifier()
model.fit(new_new_new_X, y)

/opt/homebrew/Caskroom/miniforge/base/envs/MLOps/lib/python3.8/site-packages/sklearn/base.py:1152: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


RandomForestClassifier()

In [46]:
fitted_value = model.predict(new_new_new_X)

In [47]:
accuracy_score(fitted_value, y)

0.9988776655443322

# 9. test set 예측 

In [48]:
new_test = enc.transform(test[cat_cols])

In [49]:
new_new_test = np.concatenate([new_test, test[num_cols].to_numpy()], axis=1)

In [50]:
new_new_new_test = imp.transform(new_new_test)

# 10. test set 예측값 도출 #

In [51]:
test_pred = model.predict(new_new_new_test)

In [52]:
test_pred

array([0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1,
       1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1,
       0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0,
       1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1,
       1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0,
       0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0,
       1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1,
       0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,